In [1]:
"""
Empty workspace to test and learn things
"""

import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_datasets as tfds
import keras.api._v2.keras as keras
import pandas as pd
import numpy as np
from typing import Dict, Text


# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")

# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

/Users/nick/miniforge3/envs/recSysEnv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(type(movies))

<class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>


In [11]:
ratings2 = ratings.map(lambda x : {
    'movie_title' : x['movie_title'],
    'user_id' : x['user_id'],
    'user_rating' : x['user_rating']
})
movies2 = movies.map(lambda x : x['movie_title'])


In [9]:
print(ratings)

<_PrefetchDataset element_spec={'bucketized_user_age': TensorSpec(shape=(), dtype=tf.float32, name=None), 'movie_genres': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'movie_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'raw_user_age': TensorSpec(shape=(), dtype=tf.float32, name=None), 'timestamp': TensorSpec(shape=(), dtype=tf.int64, name=None), 'user_gender': TensorSpec(shape=(), dtype=tf.bool, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_occupation_label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'user_occupation_text': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(), dtype=tf.float32, name=None), 'user_zip_code': TensorSpec(shape=(), dtype=tf.string, name=None)}>


In [15]:
filtered_ratings = ratings.filter(lambda x: x["user_id"] == '138')

# Iterate and print each row for the specified user ID
for rating in filtered_ratings:
    print(rating)

{'bucketized_user_age': <tf.Tensor: shape=(), dtype=float32, numpy=45.0>, 'movie_genres': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([7])>, 'movie_id': <tf.Tensor: shape=(), dtype=string, numpy=b'357'>, 'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b"One Flew Over the Cuckoo's Nest (1975)">, 'raw_user_age': <tf.Tensor: shape=(), dtype=float32, numpy=46.0>, 'timestamp': <tf.Tensor: shape=(), dtype=int64, numpy=879024327>, 'user_gender': <tf.Tensor: shape=(), dtype=bool, numpy=True>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'138'>, 'user_occupation_label': <tf.Tensor: shape=(), dtype=int64, numpy=4>, 'user_occupation_text': <tf.Tensor: shape=(), dtype=string, numpy=b'doctor'>, 'user_rating': <tf.Tensor: shape=(), dtype=float32, numpy=4.0>, 'user_zip_code': <tf.Tensor: shape=(), dtype=string, numpy=b'53211'>}
{'bucketized_user_age': <tf.Tensor: shape=(), dtype=float32, numpy=45.0>, 'movie_genres': <tf.Tensor: shape=(3,), dtype=int64, numpy=array([ 0,  4, 19])

In [13]:
counter = 0
for title in ratings2:
    print(title)
    counter += 1
    if counter == 5:
        break
print(ratings2['user_id'] == '138')

{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b"One Flew Over the Cuckoo's Nest (1975)">, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'138'>, 'user_rating': <tf.Tensor: shape=(), dtype=float32, numpy=4.0>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Strictly Ballroom (1992)'>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'92'>, 'user_rating': <tf.Tensor: shape=(), dtype=float32, numpy=2.0>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Very Brady Sequel, A (1996)'>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'301'>, 'user_rating': <tf.Tensor: shape=(), dtype=float32, numpy=4.0>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Pulp Fiction (1994)'>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'60'>, 'user_rating': <tf.Tensor: shape=(), dtype=float32, numpy=4.0>}
{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Scream 2 (1997)'>, 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'

2024-02-12 15:44:37.853047: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


TypeError: '_MapDataset' object is not subscriptable

##### Side Note
We want the dataset to be in the format such that every users interaction with an item can be found in a row with their context features alongside article and rating. 

##### Implementation
Regarding implementation this would mean melting the history / positive impressions column and keeping all other latent features as ID-variables. Before this can even happen however user histories / positive impressions need to be added into a list of values. But then in this case can melt even be used? It seems as though melt can be used in this case. Therefore maybe a new dataframe needs to be constructed iteratively (annoyingly). 

This probably means grouping by user, and creating a dictionary with userIds, and all associated information iteratively. Which should take around 20 minutes ish, since we have to iterate through history and impressions. And in that case we probably want an impression type column as well. And its going to be massive :<. 

In [ ]:
import ExploratoryAnalysis.data_processing_modules as dpm

In [ ]:
def decompose_interactions(num_iterations : int, news : pd.DataFrame, behaviors : pd.DataFrame) -> pd.DataFrame:
    """
    Iterates through user interactions creating new rows for a tensorflow compatible dataset.
    
    Args:
        num_iterations (int) : The number of rows in the resulting dataframe.

    Returns:
        results (pd.DataFrame) : The dataframe containing users and their rankings. The columns of the dataframe are
        user_id, article_id, rating, rating_type, category, sub_category, timestamp. 
    impression_id,user_id,time,history,impressions
    news_id,category,sub_category,title,abstract,url,title_entities,abstract_entities
    """
    data = {'user_id' : [], 'time' : [], 'news_id' : [], 'category' : [], 'sub_category' : [], 'title' : [], 'abstract' : [], 'interaction_type' : []}

    # Setting the index of news to news_id so that lookup for information can be done.
    news.set_index('news_id', inplace=True)

    # Creating an update_data function to better manage boilerplate.
    def update_data(data : dict, user_id : str, time_stamp : str, news_id : str, interaction_type : str) -> dict:
        """
        Updates the data dictionary with a users interaciton data.
        """
        # Updating all relevant keys in the dictionary
        data['user_id'].append(user_id)
        data['time'].append(time_stamp)
        data['news_id'].append(news_id)
        data['category'].append(news.loc[news_id]['category'])
        data['sub_category'].append(news.loc[news_id]['sub_category'])
        data['title'].append(news.loc[news_id]['title'])
        data['abstract'].append(news.loc[news_id]['abstract'])
        data['interaction_type'].append(interaction_type)
        return data

    # Initializing a counter so that the number of new rows can be controlled for easier testing and processing.
    counter = 0

    # Iterating through all relevant information.
    for user_id, history, impressions, time_stamp in zip(behaviors['user_id'], behaviors['history'], behaviors['impressions'], behaviors['history']):
        if counter > num_iterations:
            break

        # Iterating through all news ids in the history and impressions.
        for news_id in history.split():
            data = update_data(data, user_id, time_stamp, news_id, 'history')
            counter += 1
        for impression in impressions.split():
            impression_result = clean_impression(impression)
            if impression_result['score'] == 1:
                data = update_data(data, user_id, time_stamp, impression_result['article_ID'], 'impression')   
                counter += 1

    # Returning a dataframe with the dictionary as its input.
    return pd.DataFrame(data=data)
    
        
